In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import streamlit as st

def process_cot_data(df):
    """Transform raw COT data into positioning metrics"""
    # Convert date and numeric fields
    df['Report_Date_as_YYYY-MM-DD'] = pd.to_datetime(df['Report_Date_as_YYYY-MM-DD'])
    numeric_cols = ['Open_Interest_All', 'M_Money_Positions_Long_All', 
                   'M_Money_Positions_Short_All', 'Prod_Merc_Positions_Long_All',
                   'Prod_Merc_Positions_Short_All']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    # Calculate net positions
    df['MM_Net'] = df['M_Money_Positions_Long_All'] - df['M_Money_Positions_Short_All']
    df['Comm_Net'] = df['Prod_Merc_Positions_Long_All'] - df['Prod_Merc_Positions_Short_All']
    
    # Calculate as % of Open Interest (using pre-calculated columns where available)
    df['MM_Pct_OI'] = (df['MM_Net'] / df['Open_Interest_All']) * 100
    df['Comm_Pct_OI'] = (df['Comm_Net'] / df['Open_Interest_All']) * 100
    
    # Extract commodity name
    df['Commodity'] = df['Market_and_Exchange_Names'].str.split(' - ').str[0]
    
    return df[[
        'Report_Date_as_YYYY-MM-DD', 'Commodity', 'Open_Interest_All',
        'MM_Net', 'MM_Pct_OI', 'Comm_Net', 'Comm_Pct_OI',
        'Pct_of_OI_M_Money_Long_All', 'Pct_of_OI_M_Money_Short_All',
        'Pct_of_OI_Prod_Merc_Long_All', 'Pct_of_OI_Prod_Merc_Short_All'
    ]].dropna()

def calculate_extremes(processed_df):
    """Calculate percentile ranks and alert flags with trader concentration"""
    results = []
    for commodity, group in processed_df.groupby('Commodity'):
        # Managed Money metrics
        current_mm = group['MM_Pct_OI'].iloc[-1]
        hist_mm = group['MM_Pct_OI'].iloc[:-1]  # Exclude current week
        mm_percentile = stats.percentileofscore(hist_mm, current_mm)
        
        # Commercial hedging context
        current_comm = group['Comm_Pct_OI'].iloc[-1]
        comm_rank = stats.percentileofscore(
            group['Comm_Pct_OI'].iloc[:-1], 
            current_comm
        )
        
        # Trader concentration (using the concentration columns)
        trader_concentration = group['Conc_Gross_LE_8_TDR_Long_All'].mean()  # Example metric
        
        results.append({
            'Commodity': commodity,
            'Current_MM_Pct': current_mm,
            'MM_Percentile': mm_percentile,
            'Current_Comm_Pct': current_comm,
            'Comm_Rank': comm_rank,
            'Open_Interest': group['Open_Interest_All'].iloc[-1],
            'Trader_Concentration': trader_concentration,
            'MM_Long_Pct': group['Pct_of_OI_M_Money_Long_All'].iloc[-1],
            'MM_Short_Pct': group['Pct_of_OI_M_Money_Short_All'].iloc[-1],
            'Last_Date': group['Report_Date_as_YYYY-MM-DD'].iloc[-1].strftime('%Y-%m-%d')
        })
    
    alert_df = pd.DataFrame(results)
    
    # Flag extremes with concentration context
    conditions = [
        (alert_df['MM_Percentile'] >= 90) & (alert_df['Trader_Concentration'] > 30,
        (alert_df['MM_Percentile'] <= 10) & (alert_df['Trader_Concentration'] > 30,
        (alert_df['MM_Percentile'] >= 90),
        (alert_df['MM_Percentile'] <= 10)
    ]
    choices = [
        '🔴🚨 Extreme Overbought (Concentrated)',
        '🟢🚨 Extreme Oversold (Concentrated)',
        '🔴 Overbought',
        '🟢 Oversold'
    ]
    alert_df['Alert'] = np.select(conditions, choices, default='⚪️ Neutral')
    
    return alert_df.sort_values('MM_Percentile', ascending=False)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 70 (3179191149.py, line 73)